### RiskCalc examples.
Make sure that the working directory for this ipynb notebook is ./risktables/risktables

___
The class ```RiskCalcs``` performs basic portfolio risk calculations.  It returns a dictionary, whose items are either instances of Pandas ```DataFrame``` or floating point values representing portfolio aggregate risk measurements (VaR, Greeks, etc).
___

In [68]:
# include both the project package, and the project folder in sys.path
import sys,os
if  not './' in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not '../' in sys.path:
    sys.path.append(os.path.abspath('../'))

from risktables import risk_tables
import pandas as pd
from IPython import display
import datetime
import financialmodelingprep as fprep

#### Create an instance of RiskCalcs

In [ ]:
rt = risk_tables.RiskCalcs(use_postgres=False)

#### Create an example portfolio consisting of hypothetical options on the S&P 500 Sector Spdr's

In [ ]:
next_year = (datetime.datetime.now() + datetime.timedelta(weeks=52)).year
next_year = str(next_year)
df_spdr_options =  pd.read_csv('spdr_stocks.csv')
df_spdr_options.symbol = df_spdr_options.symbol.apply(lambda s: s.replace('2021',next_year))
display.display(df_spdr_options)


#### Run risk calculations
*Market data must be fetched for each underlying, which causes the cell below to take about 10 seconds to run*

In [ ]:
rt = risk_tables.RiskCalcs(use_postgres=False)
risk_dictionary = rt.calculate(df_spdr_options)

#### List the names of the outputs

In [ ]:
for k in risk_dictionary.keys():
    print(k)

#### Now show the actual values of each key

In [ ]:
for k in risk_dictionary.keys():
    print(f'********************* {k} **************************')
    dict_item = risk_dictionary[k]
    object_to_print = dict_item
    if type(dict_item)==dict:
        if k[:2] == 'df':
            object_to_print = risk_tables.make_df(risk_dictionary[k])
    display.display(object_to_print)


### Run the risk server on a specific port using uvicorn
1. Run the 2 cells below
2. To see an example VaR output, in a browser's address bar enter:
  * if the port is 8556: `http://localhost:8556/get_var
3. To see an example of getting risk information for the example portfolio in spdr_stocks..csv:
  * if the port is 8556: `http://localhost:8556/get_risk

In [ ]:
port_num = 8556
!uvicorn risk_server:app --port {port_num} --reload

### Show basic usage of `build_history.HistoryBuilder()`
`build_history.HistoryBuilder` has a member method `build_history_dict()`, which will create a python dictionary of daily historical values from various sources.  The default source is `yfinance` (yahoo finance)

In [2]:
from risktables import build_history as bhist
hist_builder = bhist.HistoryBuilder()
hist_dict = hist_builder.build_history_dict()

fetching sp constituents ...


### Show how to access the redis server, and retrieve objects from it like pandas DataFrames

In [16]:
import redis
import pyarrow as pa
redis_port = 6379
redis_db = redis.Redis(host = 'localhost',port=redis_port,db=0)


In [17]:
def get_redis_df(key):
    context = pa.default_serialization_context()#@UndefinedVariable 
    df = context.deserialize(redis_db.get(key))
    return df

In [19]:
redis_db.keys()

[b'QQQ_csv',
 b'PRU_csv',
 b'JBHT_csv',
 b'MKC_csv',
 b'ORLY_csv',
 b'MPC_csv',
 b'JNJ_csv',
 b'REG_csv',
 b'PAYX_csv',
 b'FIS_csv',
 b'MSFT_csv',
 b'TWTR_csv',
 b'PSX_csv',
 b'MDT_csv',
 b'SYK_csv',
 b'FLS_csv',
 b'GILD_csv',
 b'LIN_csv',
 b'NEM_csv',
 b'IDXX_csv',
 b'ANSS_csv',
 b'TXN_csv',
 b'UAL_csv',
 b'DTE_csv',
 b'LLY_csv',
 b'GE_csv',
 b'DISCK_csv',
 b'SRE_csv',
 b'FMC_csv',
 b'RCL_csv',
 b'QRVO_csv',
 b'CF_csv',
 b'AJG_csv',
 b'EFX_csv',
 b'LYV_csv',
 b'CPB_csv',
 b'CAG_csv',
 b'ES_csv',
 b'XLV_csv',
 b'BIIB_csv',
 b'IBM_csv',
 b'CMA_csv',
 b'HSIC_csv',
 b'df_tnx',
 b'PKG_csv',
 b'WMB_csv',
 b'RE_csv',
 b'ABMD_csv',
 b'META_csv',
 b'AME_csv',
 b'PXD_csv',
 b'BWA_csv',
 b'IP_csv',
 b'AVB_csv',
 b'PG_csv',
 b'ETR_csv',
 b'STT_csv',
 b'POOL_csv',
 b'J_csv',
 b'LDOS_csv',
 b'ZBH_csv',
 b'ISRG_csv',
 b'TPR_csv',
 b'AAP_csv',
 b'SBAC_csv',
 b'NVR_csv',
 b'CRM_csv',
 b'DVA_csv',
 b'PNR_csv',
 b'GM_csv',
 b'STE_csv',
 b'XEL_csv',
 b'NRG_csv',
 b'ADI_csv',
 b'CHD_csv',
 b'LNT_csv',
 b'

In [69]:
stocks = ['SPY','IBM']
dfp = fprep.fmp_profile(stocks)
dfr = fprep.fmp_ratios(stocks)


In [73]:
dfr[dfr.symbol=='SPY']

,date,investmentValuationRatios.priceBookValueRatio,investmentValuationRatios.priceToBookRatio,investmentValuationRatios.priceToSalesRatio,investmentValuationRatios.priceEarningsRatio,investmentValuationRatios.receivablesTurnover,investmentValuationRatios.priceToFreeCashFlowsRatio,investmentValuationRatios.priceToOperatingCashFlowsRatio,investmentValuationRatios.priceCashFlowRatio,investmentValuationRatios.priceEarningsToGrowthRatio,...,cashFlowIndicatorRatios.payoutRatio,cashFlowIndicatorRatios.receivablesTurnover,cashFlowIndicatorRatios.operatingCashFlowSalesRatio,cashFlowIndicatorRatios.freeCashFlowOperatingCashFlowRatio,cashFlowIndicatorRatios.cashFlowCoverageRatios,cashFlowIndicatorRatios.shortTermCoverageRatios,cashFlowIndicatorRatios.capitalExpenditureCoverageRatios,cashFlowIndicatorRatios.dividendpaidAndCapexCoverageRatios,cashFlowIndicatorRatios.dividendPayoutRatio,symbol
